In [1]:
import cv2
from picamera2 import Picamera2
from qreader import QReader
import parking_slots
import json
import os

In [ ]:
picam2 = Picamera2()
picam2.preview_configuration.main.size = (3000,3000)
picam2.preview_configuration.main.format = "RGB888"
qreader_reader = QReader()

current_fleet = []

In [3]:
def load_parking_slots(json_file):
    with open(json_file, "r") as f:
        slots = json.load(f)
    return [(slot["coordinates"], slot["slot_id"]) for slot in slots]

In [4]:
def capture_frame():
    picam2.start()

    frame = picam2.capture_array()

    output_path = "Testaufnahme_RGB.png"

    cv2.imwrite(output_path, frame)

    picam2.stop()

    return frame

In [5]:
def detectQRCodes():
    img_path = "Testaufnahme_RGB.png"
    img = cv2.imread(img_path)
    img = capture_frame()
    slots,image = load_parking_slots("parking_slots.json"),cv2.imread("/home/raspi5/Desktop/Muhammad/qreader_test_image_detections.png")  #parking_slots.define_parking_slots(img)

    decoded_texts, detections = qreader_reader.detect_and_decode(image=image,return_detections=True)
    
    for decoded_text, detection in zip(decoded_texts,detections):
        x1,y1,x2,y2 = map(int, detection['bbox_xyxy'])
        qr_code_center = ((x1 + x2) // 2, (y1 + y2) // 2)
        for s in slots:
            coordinates,slot_id=s
            if parking_slots.is_point_in_parking_slot(qr_code_center,coordinates):
                print(f"Vehicle {decoded_text} is on park slot id {slot_id}")

        cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=4)
        cv2.putText(image, f'{decoded_text}', (x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=2, color=(0, 0, 2550), thickness=4)
        
    cv2.imwrite(filename='qreader_test_image_detections.png', img=img)

    print(f"img: {img_path} -> QReader: {decoded_texts}") 

In [ ]:
detectQRCodes()